# Morris散点图

## 原图

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import os
import numpy as np
from adjustText import adjust_text

# 全局字体
matplotlib.rcParams['font.family'] = 'Times New Roman'
matplotlib.rcParams['font.sans-serif'] = ['SimSun', 'Times New Roman']
matplotlib.rcParams['axes.unicode_minus'] = False

# 文件路径和标签
csv_files = [
    (r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\fei_model_GCAM 5.3\1_综合比较不同敏感度指标\fei_SHAP_vs_Imp.csv", "a"),
    (r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\fet_model_GCAM 5.3\1_综合比较不同敏感度指标\fet_SHAP_vs_Imp.csv", "b"),
    (r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\pe_model_COFFEE 1.1\1_综合比较不同敏感度指标\pe_SHAP_vs_Imp.csv", "c"),
    (r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\see_model_IMACLIM 1.1\1_综合比较不同敏感度指标\see_SHAP_vs_Imp.csv", "d"),
]
outdir = r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\RF图像优化"
os.makedirs(outdir, exist_ok=True)
outpath = os.path.join(outdir, "RF_Morris_优化.png")

# 智能文本调整
def smart_adjust_text(texts, ax, max_shrinkA=27, start_shrinkA=7, step=5):
    for shrinkA in range(start_shrinkA, max_shrinkA+1, step):
        try:
            adjust_text(
                texts, ax=ax,
                expand_points=(2.8, 3.1),
                arrowprops=dict(arrowstyle="-", color="black", lw=1.25, alpha=0.82, shrinkA=shrinkA)
            )
            break
        except Exception:
            continue

# 绘制单个子图
def plot_morris(df, ax, mono_cmap, combo_exp=3.2):
    def norm(x): return (x - x.min()) / (x.max() - x.min() + 1e-8)
    norm_SHAP = norm(df["均值_SHAP"])
    norm_STD = norm(df["SHAP_标准差"])
    norm_imp = norm(df["特征重要性"])
    combo = 0.4 * norm_SHAP + 0.4 * norm_STD + 0.2 * norm_imp
    size = 180 + 9200 * (combo ** combo_exp)
    bubble_colors = mono_cmap(norm_imp)

    # 主气泡
    for i in range(len(df)):
        ax.scatter(
            df["均值_SHAP"][i], df["SHAP_标准差"][i],
            s=size[i], color=bubble_colors[i],
            alpha=0.77, linewidths=0, zorder=2
        )

    # 标签
    texts = []
    for i in range(len(df)):
        if size[i] > 2200:  # 大气泡：居中
            texts.append(ax.text(
                df["均值_SHAP"][i], df["SHAP_标准差"][i],
                str(df["变量"][i]),
                fontsize=18 + int(9 * combo[i]),
                color='black',
                ha='center', va='center', zorder=10,
                fontweight='bold', fontfamily='Times New Roman',
                bbox=dict(boxstyle="round,pad=0.18", fc="white", alpha=0.82, lw=0)
            ))
        else:  # 小气泡：相对偏移
            dx = 0.02 * (df["均值_SHAP"].max() - df["均值_SHAP"].min())
            dy = 0.02 * (df["SHAP_标准差"].max() - df["SHAP_标准差"].min())

            texts.append(ax.text(
                df["均值_SHAP"][i] + dx, df["SHAP_标准差"][i] + dy,
                str(df["变量"][i]),
                fontsize=18 + int(6 * combo[i]),
                color='black',
                ha='left', va='bottom', zorder=10,
                fontweight='bold', fontfamily='Times New Roman',
                bbox=dict(boxstyle="round,pad=0.13", fc="white", alpha=0.75, lw=0)
            ))

    smart_adjust_text(texts, ax)

    # 坐标轴 & 样式
    ax.set_ylabel("Morris σ", fontsize=30, fontweight='bold',
                  color='black', labelpad=15, fontfamily='Times New Roman')
    ax.set_xlabel("Morris µ*", fontsize=30, fontweight='bold',
                  color='black', fontfamily='Times New Roman', labelpad=11)
    ax.grid(True, linestyle="--", alpha=0.13, zorder=0)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.tick_params(axis='y', labelsize=30)
    ax.tick_params(axis='x', labelsize=30)

    # 刻度字体不加粗
    for label in ax.get_yticklabels():
        label.set_fontname('Times New Roman')
    for label in ax.get_xticklabels():
        label.set_fontname('Times New Roman')

# 绿色系 colormap
mono_cmap = plt.cm.Greens

# 2×2 布局
fig, axes = plt.subplots(2, 2, figsize=(16, 16), sharex=False, constrained_layout=False)
axes = axes.flatten()

for i, (csv_path, label) in enumerate(csv_files):
    df = pd.read_csv(csv_path, encoding="utf-8-sig")
    ax = axes[i]
    plot_morris(df, ax, mono_cmap)

    # 子图角标
    ax.text(0.01, 0.97, label, transform=ax.transAxes,
            fontsize=30, fontweight='bold', va='top', ha='left',
            color='black', fontfamily='Times New Roman')

# 色条
all_min = min(pd.read_csv(f[0], encoding="utf-8-sig")["特征重要性"].min() for f in csv_files)
all_max = max(pd.read_csv(f[0], encoding="utf-8-sig")["特征重要性"].max() for f in csv_files)
norm = plt.Normalize(vmin=all_min, vmax=all_max)
sm = plt.cm.ScalarMappable(cmap=mono_cmap, norm=norm)
sm.set_array([])

cbar = fig.colorbar(
    sm,
    ax=axes.ravel().tolist(),
    orientation='vertical',
    shrink=0.80,
    pad=0.06,
    fraction=0.037,
    aspect=22
)
cbar.set_label(
    "特征重要性",
    fontsize=30, fontweight='bold', fontfamily='SimSun', color='black', labelpad=28
)
for label in cbar.ax.get_yticklabels():
    label.set_fontname('Times New Roman')
    label.set_fontsize(30)

cbar.ax.tick_params(colors='black', labelsize=15)
for spine in cbar.ax.spines.values():
    spine.set_edgecolor('black')

# 布局优化 & 保存
plt.subplots_adjust(left=0.09, right=0.86, top=0.96, bottom=0.07,
                    hspace=0.25, wspace=0.22)
plt.savefig(outpath, dpi=340, transparent=True)
plt.close()

print(f"[✔] 2×2 Morris 气泡图（相对偏移修正版）已保存至：{outpath}")


[✔] 2×2 Morris 气泡图（相对偏移修正版）已保存至：C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\RF图像优化\RF_Morris_优化.png


## 去掉IMACLIM 1.1

In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import os
import numpy as np
from adjustText import adjust_text

# 全局字体
matplotlib.rcParams['font.family'] = 'Times New Roman'
matplotlib.rcParams['font.sans-serif'] = ['SimSun', 'Times New Roman']
matplotlib.rcParams['axes.unicode_minus'] = False

# 文件路径和标签
csv_files = [
    (r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\fei_model_GCAM 5.3\1_综合比较不同敏感度指标\fei_SHAP_vs_Imp.csv", "a"),
    (r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\fet_model_GCAM 5.3\1_综合比较不同敏感度指标\fet_SHAP_vs_Imp.csv", "b"),
    (r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\pe_model_COFFEE 1.1\1_综合比较不同敏感度指标\pe_SHAP_vs_Imp.csv", "c"),
]

# 输出文件夹和文件名
outdir = r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\RF图像优化\去除IMACLIM1.1 SEE"
os.makedirs(outdir, exist_ok=True)
outpath = os.path.join(outdir, "RF_Morris优化_去IMACLIM 1.1.png")

# --- 修改点: 提高 shrinkA 的起始值以优化箭头/连接线 ---
def smart_adjust_text(texts, ax, max_shrinkA=27, start_shrinkA=12, step=5): # start_shrinkA 从 7 改为 12
    for shrinkA in range(start_shrinkA, max_shrinkA+1, step):
        try:
            adjust_text(
                texts, ax=ax,
                expand_points=(2.8, 3.1),
                arrowprops=dict(arrowstyle="-", color="black", lw=1.25, alpha=0.82, shrinkA=shrinkA)
            )
            break
        except Exception:
            continue

# 绘制单个子图
def plot_morris(df, ax, mono_cmap, combo_exp=3.2):
    def norm(x): return (x - x.min()) / (x.max() - x.min() + 1e-8)
    norm_SHAP = norm(df["均值_SHAP"])
    norm_STD = norm(df["SHAP_标准差"])
    norm_imp = norm(df["特征重要性"])
    combo = 0.4 * norm_SHAP + 0.4 * norm_STD + 0.2 * norm_imp
    size = 180 + 13000 * (combo ** combo_exp)
    bubble_colors = mono_cmap(norm_imp)

    # 主气泡
    for i in range(len(df)):
        ax.scatter(
            df["均值_SHAP"][i], df["SHAP_标准差"][i],
            s=size[i], color=bubble_colors[i],
            alpha=0.77, linewidths=0, zorder=2
        )

    # 标签
    texts = []
    for i in range(len(df)):
        if size[i] > 2200:  # 大气泡：居中
            texts.append(ax.text(
                df["均值_SHAP"][i], df["SHAP_标准差"][i],
                str(df["变量"][i]),
                fontsize=22 + int(15 * combo[i]),
                color='black',
                ha='center', va='center', zorder=10,
                fontweight='bold', fontfamily='Times New Roman',
                bbox=dict(boxstyle="round,pad=0.18", fc="white", alpha=0.82, lw=0)
            ))
        else:  # 小气泡：相对偏移
            dx = 0.02 * (df["均值_SHAP"].max() - df["均值_SHAP"].min())
            dy = 0.02 * (df["SHAP_标准差"].max() - df["SHAP_标准差"].min())

            texts.append(ax.text(
                df["均值_SHAP"][i] + dx, df["SHAP_标准差"][i] + dy,
                str(df["变量"][i]),
                fontsize=22 + int(15 * combo[i]),
                color='black',
                ha='left', va='bottom', zorder=10,
                fontweight='bold', fontfamily='Times New Roman',
                bbox=dict(boxstyle="round,pad=0.13", fc="white", alpha=0.75, lw=0)
            ))

    smart_adjust_text(texts, ax)

    # 坐标轴 & 样式
    ax.set_ylabel("Morris σ", fontsize=40, fontweight='bold',
                  color='black', labelpad=15, fontfamily='Times New Roman')
    ax.set_xlabel("Morris µ*", fontsize=40, fontweight='bold',
                  color='black', fontfamily='Times New Roman', labelpad=11)
    ax.grid(True, linestyle="--", alpha=0.13, zorder=0)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.tick_params(axis='y', labelsize=30)
    ax.tick_params(axis='x', labelsize=30)

    # 刻度字体不加粗
    for label in ax.get_yticklabels():
        label.set_fontname('Times New Roman')
    for label in ax.get_xticklabels():
        label.set_fontname('Times New Roman')

# 绿色系 colormap
mono_cmap = plt.cm.Greens

# 2×2 布局
fig, axes = plt.subplots(2, 2, figsize=(16, 20), sharex=False, constrained_layout=False)
axes = axes.flatten()

for i, (csv_path, label) in enumerate(csv_files):
    df = pd.read_csv(csv_path, encoding="utf-8-sig")
    ax = axes[i]
    plot_morris(df, ax, mono_cmap)

    # 子图角标
    ax.text(0.01, 0.97, label, transform=ax.transAxes,
            fontsize=40, fontweight='bold', va='top', ha='left',
            color='black', fontfamily='Times New Roman')

# 隐藏2x2布局中未使用的子图
for i in range(len(csv_files), len(axes)):
    axes[i].set_visible(False)

# 色条
all_min = min(pd.read_csv(f[0], encoding="utf-8-sig")["特征重要性"].min() for f in csv_files)
all_max = max(pd.read_csv(f[0], encoding="utf-8-sig")["特征重要性"].max() for f in csv_files)
norm = plt.Normalize(vmin=all_min, vmax=all_max)
sm = plt.cm.ScalarMappable(cmap=mono_cmap, norm=norm)
sm.set_array([])

cbar = fig.colorbar(
    sm,
    ax=axes.ravel().tolist(),
    orientation='vertical',
    shrink=0.80,
    pad=0.06,
    fraction=0.037,
    aspect=22
)
cbar.set_label(
    "特征重要性",
    fontsize=35, fontweight='bold', fontfamily='SimSun', color='black', labelpad=28
)
for label in cbar.ax.get_yticklabels():
    label.set_fontname('Times New Roman')
    label.set_fontweight('bold')

cbar.ax.tick_params(colors='black', labelsize=25)
for spine in cbar.ax.spines.values():
    spine.set_edgecolor('black')

# 布局优化 & 保存
plt.subplots_adjust(left=0.09, right=0.86, top=0.96, bottom=0.07,
                    hspace=0.25, wspace=0.22)
plt.savefig(outpath, dpi=340, transparent=True)
plt.close()

print(f"[✔] 图像已成功保存至：{outpath}")

[✔] 图像已成功保存至：C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\RF图像优化\去除IMACLIM1.1 SEE\RF_Morris优化_去IMACLIM 1.1.png


### 修改

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import os
import numpy as np
from adjustText import adjust_text

# --- 修改：全局字体设置，移除SimSun，确保默认使用Times New Roman ---
matplotlib.rcParams['font.family'] = 'Times New Roman'
# matplotlib.rcParams['font.sans-serif'] = ['SimSun', 'Times New Roman'] # 移除SimSun
matplotlib.rcParams['axes.unicode_minus'] = False

# 文件路径和标签 (无变化)
csv_files = [
    (r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\fei_model_GCAM 5.3\1_综合比较不同敏感度指标\fei_SHAP_vs_Imp.csv", "a"),
    (r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\fet_model_GCAM 5.3\1_综合比较不同敏感度指标\fet_SHAP_vs_Imp.csv", "b"),
    (r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\pe_model_COFFEE 1.1\1_综合比较不同敏感度指标\pe_SHAP_vs_Imp.csv", "c"),
]

# 输出文件夹和文件名 (无变化)
outdir = r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\RF图像优化\去除IMACLIM1.1 SEE"
os.makedirs(outdir, exist_ok=True)
outpath = os.path.join(outdir, "RF_Morris优化_去IMACLIM 1.1_Vertical.png")

# --- 文本标签优化函数 (无变化) ---
def smart_adjust_text(texts, ax, max_shrinkA=27, start_shrinkA=12, step=5):
    for shrinkA in range(start_shrinkA, max_shrinkA+1, step):
        try:
            adjust_text(
                texts, ax=ax,
                expand_points=(2.8, 3.1),
                arrowprops=dict(arrowstyle="-", color="black", lw=1.25, alpha=0.82, shrinkA=shrinkA)
            )
            break
        except Exception:
            continue

# --- 绘制单个子图函数 (修改内部字体设置) ---
def plot_morris(df, ax, mono_cmap, combo_exp=3.2):
    def norm(x): return (x - x.min()) / (x.max() - x.min() + 1e-8)
    norm_SHAP = norm(df["均值_SHAP"])
    norm_STD = norm(df["SHAP_标准差"])
    norm_imp = norm(df["特征重要性"])
    combo = 0.4 * norm_SHAP + 0.4 * norm_STD + 0.2 * norm_imp
    size = 180 + 13000 * (combo ** combo_exp)
    bubble_colors = mono_cmap(norm_imp)

    # 主气泡 (无变化)
    for i in range(len(df)):
        ax.scatter(
            df["均值_SHAP"][i], df["SHAP_标准差"][i],
            s=size[i], color=bubble_colors[i],
            alpha=0.77, linewidths=0, zorder=2
        )

    # 标签 (字体已是Times New Roman, bold，无需修改)
    texts = []
    for i in range(len(df)):
        if size[i] > 2200:  # 大气泡：居中
            texts.append(ax.text(
                df["均值_SHAP"][i], df["SHAP_标准差"][i],
                str(df["变量"][i]),
                fontsize=35 + int(25 * combo[i]),
                color='black',
                ha='center', va='center', zorder=10,
                fontweight='bold', fontfamily='Times New Roman',
                bbox=dict(boxstyle="round,pad=0.18", fc="white", alpha=0.82, lw=0)
            ))
        else:  # 小气泡：相对偏移
            dx = 0.02 * (df["均值_SHAP"].max() - df["均值_SHAP"].min())
            dy = 0.02 * (df["SHAP_标准差"].max() - df["SHAP_标准差"].min())
            texts.append(ax.text(
                df["均值_SHAP"][i] + dx, df["SHAP_标准差"][i] + dy,
                str(df["变量"][i]),
                fontsize=28 + int(20 * combo[i]),
                color='black',
                ha='left', va='bottom', zorder=10,
                fontweight='bold', fontfamily='Times New Roman',
                bbox=dict(boxstyle="round,pad=0.13", fc="white", alpha=0.75, lw=0)
            ))
    smart_adjust_text(texts, ax)

    # 坐标轴 & 样式 (轴标签已是Times New Roman, bold，无需修改)
    ax.set_ylabel("Morris σ", fontsize=50, fontweight='bold',
                  color='black', labelpad=15, fontfamily='Times New Roman')
    ax.set_xlabel("Morris µ*", fontsize=50, fontweight='bold',
                  color='black', fontfamily='Times New Roman', labelpad=11)
    ax.grid(True, linestyle="--", alpha=0.13, zorder=0)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.tick_params(axis='y', labelsize=30)
    ax.tick_params(axis='x', labelsize=30)

    # --- 修改：将坐标轴刻度字体设置为 Times New Roman 并加粗 ---
    for label in ax.get_yticklabels():
        label.set_fontname('Times New Roman')
        label.set_fontweight('bold') # 加粗
    for label in ax.get_xticklabels():
        label.set_fontname('Times New Roman')
        label.set_fontweight('bold') # 加粗
        
# 绿色系 colormap (无变化)
mono_cmap = plt.cm.Greens

# 3行1列布局 (无变化)
fig, axes = plt.subplots(3, 1, figsize=(24, 30))
axes = axes.flatten()

# 循环绘制子图
for i, (csv_path, label) in enumerate(csv_files):
    df = pd.read_csv(csv_path, encoding="utf-8-sig")
    ax = axes[i]
    plot_morris(df, ax, mono_cmap)

    # 子图角标 (字体已是Times New Roman, bold，无需修改)
    ax.text(0.02, 0.96, label, transform=ax.transAxes,
            fontsize=50, fontweight='bold', va='top', ha='left',
            color='black', fontfamily='Times New Roman')

# 计算所有数据中的最大最小值 (无变化)
all_min = min(pd.read_csv(f[0], encoding="utf-8-sig")["特征重要性"].min() for f in csv_files)
all_max = max(pd.read_csv(f[0], encoding="utf-8-sig")["特征重要性"].max() for f in csv_files)
norm = plt.Normalize(vmin=all_min, vmax=all_max)
sm = plt.cm.ScalarMappable(cmap=mono_cmap, norm=norm)
sm.set_array([])

# 色条参数 (无变化)
cbar = fig.colorbar(
    sm,
    ax=axes.ravel().tolist(),
    orientation='vertical',
    shrink=0.6,
    pad=0.03,
    fraction=0.04,
    aspect=25
)
# --- 修改：移除色条标签文本 ---
cbar.set_label(
    "", # 将 "特征重要性" 变为空字符串
    fontsize=45, fontweight='bold', fontfamily='Times New Roman', color='black', labelpad=28
)
# 色条刻度字体设置 (已是Times New Roman, bold，无需修改)
for label in cbar.ax.get_yticklabels():
    label.set_fontname('Times New Roman')
    label.set_fontweight('bold')

cbar.ax.tick_params(colors='black', labelsize=32)
for spine in cbar.ax.spines.values():
    spine.set_edgecolor('black')

# 整体布局参数 (无变化)
plt.subplots_adjust(left=0.15, right=0.85, top=0.97, bottom=0.05, hspace=0.2)
plt.savefig(outpath, dpi=600, transparent=True, bbox_inches='tight')
plt.close()

print(f"[✔] 图像已成功保存至：{outpath}")

[✔] 图像已成功保存至：C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\RF图像优化\去除IMACLIM1.1 SEE\RF_Morris优化_去IMACLIM 1.1_Vertical.png


# SHAP依赖图

## 原图

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import os
from xgboost import XGBRegressor
from matplotlib.colors import Normalize

# 全局字体设定
matplotlib.rcParams['font.family'] = 'Times New Roman'
matplotlib.rcParams['axes.unicode_minus'] = False

# 子图配置
plots_matrix = [
    {   # 1st row: GCAM 5.3 FEI
        "group": "GCAM 5.3 FEI", "cmap": plt.cm.Reds, "plots": [
            {"csv": r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\fei_model_GCAM 5.3\2_确认非线性与变量交互\fei_SHAP_dependence_seebwoc.csv", "x": "seebwoc", "y": "SHAP_seebwoc", "color": "eced", "xlabel": "seebwoc", "inter_var": "eced"},
            {"csv": r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\fei_model_GCAM 5.3\2_确认非线性与变量交互\fei_SHAP_dependence_seec.csv", "x": "seec", "y": "SHAP_seec", "color": "eced", "xlabel": "seec", "inter_var": "eced"},
            {"csv": r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\fei_model_GCAM 5.3\2_确认非线性与变量交互\fei_SHAP_dependence_seegwoc.csv", "x": "seegwoc", "y": "SHAP_seegwoc", "color": "fel", "xlabel": "seegwoc", "inter_var": "fel"},
        ]
    },
    {   # 2nd row: GCAM 5.3 FET
        "group": "GCAM 5.3 FET", "cmap": plt.cm.Greens, "plots": [
            {"csv": r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\fet_model_GCAM 5.3\2_确认非线性与变量交互\fet_SHAP_dependence_feg.csv", "x": "feg", "y": "SHAP_feg", "color": "fel", "xlabel": "feg", "inter_var": "fel"},
            {"csv": r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\fet_model_GCAM 5.3\2_确认非线性与变量交互\fet_SHAP_dependence_fel.csv", "x": "fel", "y": "SHAP_fel", "color": "sees", "xlabel": "fel", "inter_var": "sees"},
            {"csv": r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\fet_model_GCAM 5.3\2_确认非线性与变量交互\fet_SHAP_dependence_seeb.csv", "x": "seeb", "y": "SHAP_seeb", "color": "feg", "xlabel": "seeb", "inter_var": "feg"},
        ]
    },
    {   # 3rd row: COFFEE 1.1 PE
        "group": "COFFEE 1.1 PE", "cmap": plt.cm.Purples, "plots": [
            {"csv": r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\pe_model_COFFEE 1.1\2_确认非线性与变量交互\pe_SHAP_dependence_emcoen.csv", "x": "emcoen", "y": "SHAP_emcoen", "color": "fee", "xlabel": "emcoen", "inter_var": "fee"},
            {"csv": r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\pe_model_COFFEE 1.1\2_确认非线性与变量交互\pe_SHAP_dependence_fee.csv", "x": "fee", "y": "SHAP_fee", "color": "see", "xlabel": "fee", "inter_var": "see"},
            {"csv": r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\pe_model_COFFEE 1.1\2_确认非线性与变量交互\pe_SHAP_dependence_see.csv", "x": "see", "y": "SHAP_see", "color": "seeh", "xlabel": "see", "inter_var": "seeh"},
        ]
    },
    {   # 4th row: IMACLIM SEE
        "group": "IMACLIM SEE", "cmap": plt.cm.Oranges, "plots": [
            {"csv": r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\see_model_IMACLIM 1.1\2_确认非线性与变量交互\see_SHAP_dependence_fee.csv", "x": "fee", "y": "SHAP_fee", "color": "peowoc", "xlabel": "fee", "inter_var": "peowoc"},
            {"csv": r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\see_model_IMACLIM 1.1\2_确认非线性与变量交互\see_SHAP_dependence_fet.csv", "x": "fet", "y": "SHAP_fet", "color": "pe", "xlabel": "fet", "inter_var": "pe"},
            {"csv": r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\see_model_IMACLIM 1.1\2_确认非线性与变量交互\see_SHAP_dependence_penr.csv", "x": "penr", "y": "SHAP_penr", "color": "peowoc", "xlabel": "penr", "inter_var": "peowoc"},
        ]
    }
]

outdir = r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\RF图像优化"
os.makedirs(outdir, exist_ok=True)
outpath = os.path.join(outdir, "RF_SHAP dep_up.png")

fig, axes = plt.subplots(4, 3, figsize=(18, 22), constrained_layout=True)

import string
abc_labels = list(string.ascii_lowercase[:12])  # 'a'~'l'

for row, row_info in enumerate(plots_matrix):
    for col, plot in enumerate(row_info["plots"]):
        df = pd.read_csv(plot['csv'], encoding="utf-8-sig")
        x = df[plot['x']].values.reshape(-1, 1)
        y = df[plot['y']].values
        color_val = df[plot['color']].values

        if np.nanmax(color_val) == np.nanmin(color_val):
            normed = np.full(len(color_val), 0.5)
        else:
            normed = (color_val - np.nanmin(color_val)) / (np.nanmax(color_val) - np.nanmin(color_val) + 1e-9)
            normed = np.clip(normed, 0, 1)
        color_vals_adj = 0.25 + 0.75 * normed
        colors = row_info['cmap'](color_vals_adj)
        size = 35 + (1 - normed) * 80
        size = np.where(np.isnan(size), 35, size)
        size[size <= 0] = 10

        ax = axes[row, col]
        sc = ax.scatter(
            x, y, c=colors, s=size, alpha=0.88
        )

        # XGBoost智能拟合
        try:
            xfit = np.linspace(np.nanmin(x), np.nanmax(x), 300).reshape(-1, 1)
            xgb = XGBRegressor(
                n_estimators=80, max_depth=3, learning_rate=0.07,
                random_state=2024, subsample=0.9
            )
            xgb.fit(x, y)
            yfit = xgb.predict(xfit)
            ax.plot(xfit, yfit, linestyle='--', color=row_info['cmap'](0.88), linewidth=2)
        except Exception:
            pass

        # 加大加粗横纵坐标
        ax.set_xlabel(plot['xlabel'], fontsize=30, fontweight='bold', fontname='Times New Roman')
        if col == 0:
            ax.set_ylabel("SHAP", fontsize=30, fontweight='bold', fontname='Times New Roman')
        else:
            ax.set_ylabel("")

        # 刻度字体加大加粗
        ax.tick_params(axis='both', labelsize=20)
        for label in ax.get_xticklabels() + ax.get_yticklabels():
            label.set_fontname('Times New Roman')
            label.set_fontweight('bold')

        ax.grid(True, linestyle='--', alpha=0.22, zorder=0)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)

        # ======= 色条(变量名更大更粗, 距离更近) ======
        norm = Normalize(vmin=0.25, vmax=1)
        cbar = plt.colorbar(
            plt.cm.ScalarMappable(cmap=row_info['cmap'], norm=norm),
            ax=ax, pad=0.015, shrink=0.92
        )
        cbar.set_label(plot['inter_var'],
                       fontsize=30, fontweight='bold', fontname='Times New Roman', labelpad=11)
        cbar.ax.tick_params(labelsize=17)
        for tick in cbar.ax.get_yticklabels():
            tick.set_fontname('Times New Roman')
            tick.set_fontweight('bold')
        cbar.ax.set_yticklabels([])

        # ==== 子图左上角字母 ====
        subfig_label = abc_labels[row*3 + col]
        ax.text(0.01, 0.97, subfig_label, transform=ax.transAxes,
                fontsize=22, fontweight='bold', va='top', ha='left', color='black', fontname='Times New Roman')

    # 行主标题加大
    axes[row,0].text(-0.44, 0.5, row_info['group'], transform=axes[row,0].transAxes,
                     fontsize=30, fontweight='bold', va='center', ha='right',
                     color=row_info['cmap'](0.8), rotation=90, fontname='Times New Roman')

plt.savefig(outpath, dpi=350)
plt.close()
print(f"[✔] 4×3 RF_SHAP依赖矩阵图片已保存至：{outpath}")

[✔] 4×3 RF_SHAP依赖矩阵图片已保存至：C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\RF图像优化\RF_SHAP dep_up.png


## 去掉IMACLIM 1.1 SEE

In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import os
from xgboost import XGBRegressor
from matplotlib.colors import Normalize

# 全局字体设定
matplotlib.rcParams['font.family'] = 'Times New Roman'
matplotlib.rcParams['axes.unicode_minus'] = False

# 子图配置
plots_matrix = [
    {   # 1st row: GCAM 5.3 FEI
        "group": "GCAM 5.3 FEI", "cmap": plt.cm.Reds, "plots": [
            {"csv": r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\fei_model_GCAM 5.3\2_确认非线性与变量交互\fei_SHAP_dependence_seebwoc.csv", "x": "seebwoc", "y": "SHAP_seebwoc", "color": "eced", "xlabel": "seebwoc", "inter_var": "eced"},
            {"csv": r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\fei_model_GCAM 5.3\2_确认非线性与变量交互\fei_SHAP_dependence_seec.csv", "x": "seec", "y": "SHAP_seec", "color": "eced", "xlabel": "seec", "inter_var": "eced"},
            {"csv": r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\fei_model_GCAM 5.3\2_确认非线性与变量交互\fei_SHAP_dependence_seegwoc.csv", "x": "seegwoc", "y": "SHAP_seegwoc", "color": "fel", "xlabel": "seegwoc", "inter_var": "fel"},
        ]
    },
    {   # 2nd row: GCAM 5.3 FET
        "group": "GCAM 5.3 FET", "cmap": plt.cm.Greens, "plots": [
            {"csv": r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\fet_model_GCAM 5.3\2_确认非线性与变量交互\fet_SHAP_dependence_feg.csv", "x": "feg", "y": "SHAP_feg", "color": "fel", "xlabel": "feg", "inter_var": "fel"},
            {"csv": r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\fet_model_GCAM 5.3\2_确认非线性与变量交互\fet_SHAP_dependence_fel.csv", "x": "fel", "y": "SHAP_fel", "color": "sees", "xlabel": "fel", "inter_var": "sees"},
            {"csv": r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\fet_model_GCAM 5.3\2_确认非线性与变量交互\fet_SHAP_dependence_seeb.csv", "x": "seeb", "y": "SHAP_seeb", "color": "feg", "xlabel": "seeb", "inter_var": "feg"},
        ]
    },
    {   # 3rd row: COFFEE 1.1 PE
        "group": "COFFEE 1.1 PE", "cmap": plt.cm.Purples, "plots": [
            {"csv": r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\pe_model_COFFEE 1.1\2_确认非线性与变量交互\pe_SHAP_dependence_emcoen.csv", "x": "emcoen", "y": "SHAP_emcoen", "color": "fee", "xlabel": "emcoen", "inter_var": "fee"},
            {"csv": r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\pe_model_COFFEE 1.1\2_确认非线性与变量交互\pe_SHAP_dependence_fee.csv", "x": "fee", "y": "SHAP_fee", "color": "see", "xlabel": "fee", "inter_var": "see"},
            {"csv": r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\pe_model_COFFEE 1.1\2_确认非线性与变量交互\pe_SHAP_dependence_see.csv", "x": "see", "y": "SHAP_see", "color": "seeh", "xlabel": "see", "inter_var": "seeh"},
        ]
    },
]

# 输出路径和文件名
outdir = r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\RF图像优化\去除IMACLIM1.1 SEE"
os.makedirs(outdir, exist_ok=True)
outpath = os.path.join(outdir, "RF_SHAP dep_no IMA.png")

# 创建画布
fig, axes = plt.subplots(3, 3, figsize=(18, 22), constrained_layout=True)

import string
abc_labels = list(string.ascii_lowercase[:9])  # 'a'~'i'

for row, row_info in enumerate(plots_matrix):
    # --- FIX: The line below was incomplete and caused the SyntaxError ---
    for col, plot in enumerate(row_info["plots"]):
        df = pd.read_csv(plot['csv'], encoding="utf-8-sig")
        x = df[plot['x']].values.reshape(-1, 1)
        y = df[plot['y']].values
        color_val = df[plot['color']].values

        if np.nanmax(color_val) == np.nanmin(color_val):
            normed = np.full(len(color_val), 0.5)
        else:
            normed = (color_val - np.nanmin(color_val)) / (np.nanmax(color_val) - np.nanmin(color_val) + 1e-9)
            normed = np.clip(normed, 0, 1)
        color_vals_adj = 0.25 + 0.75 * normed
        colors = row_info['cmap'](color_vals_adj)
        size = 35 + (1 - normed) * 80
        size = np.where(np.isnan(size), 35, size)
        size[size <= 0] = 10

        ax = axes[row, col]
        sc = ax.scatter(
            x, y, c=colors, s=size, alpha=0.88
        )

        # XGBoost智能拟合
        try:
            xfit = np.linspace(np.nanmin(x), np.nanmax(x), 300).reshape(-1, 1)
            xgb = XGBRegressor(
                n_estimators=80, max_depth=3, learning_rate=0.07,
                random_state=2024, subsample=0.9
            )
            xgb.fit(x, y)
            yfit = xgb.predict(xfit)
            ax.plot(xfit, yfit, linestyle='--', color=row_info['cmap'](0.88), linewidth=2)
        except Exception:
            pass

        # 加大加粗横纵坐标
        ax.set_xlabel(plot['xlabel'], fontsize=40, fontweight='bold', fontname='Times New Roman')
        if col == 0:
            ax.set_ylabel("SHAP", fontsize=35, fontweight='bold', fontname='Times New Roman')
        else:
            ax.set_ylabel("")

        # 刻度字体加大加粗
        ax.tick_params(axis='both', labelsize=20)
        for label in ax.get_xticklabels() + ax.get_yticklabels():
            label.set_fontname('Times New Roman')
            label.set_fontweight('bold')

        ax.grid(True, linestyle='--', alpha=0.22, zorder=0)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)

        # ======= 色条(变量名更大更粗, 距离更近) ======
        norm = Normalize(vmin=0.25, vmax=1)
        cbar = plt.colorbar(
            plt.cm.ScalarMappable(cmap=row_info['cmap'], norm=norm),
            ax=ax, pad=0.015, shrink=0.92
        )
        cbar.set_label(plot['inter_var'],
                       fontsize=40, fontweight='bold', fontname='Times New Roman', labelpad=11)
        cbar.ax.tick_params(labelsize=17)
        for tick in cbar.ax.get_yticklabels():
            tick.set_fontname('Times New Roman')
            tick.set_fontweight('bold')
        cbar.ax.set_yticklabels([])

        # ==== 子图左上角字母 ====
        subfig_label = abc_labels[row*3 + col]
        ax.text(0.01, 0.97, subfig_label, transform=ax.transAxes,
                fontsize=40, fontweight='bold', va='top', ha='left', color='black', fontname='Times New Roman')

    # 行主标题加大
    axes[row,0].text(-0.44, 0.5, row_info['group'], transform=axes[row,0].transAxes,
                     fontsize=40, fontweight='bold', va='center', ha='right',
                     color=row_info['cmap'](0.8), rotation=90, fontname='Times New Roman')

plt.savefig(outpath, dpi=350)
plt.close()

print(f"[✔] 3×3 RF_SHAP依赖矩阵图片已保存至：{outpath}")

[✔] 3×3 RF_SHAP依赖矩阵图片已保存至：C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\RF图像优化\去除IMACLIM1.1 SEE\RF_SHAP dep_no IMA.png
